### Práctica de Formatos de datos: XML

El objetivo de esta práctica es crear una aplicación simple para llegar determinados edificios monumentales de Madrid. Para ello se van a utilizar dos APIS diferentes ofrecidas por el portal de datos abiertos del ayuntamiento de Madrid.

En primer lugar se va a usar el catálogo "Edificios de carácter monumental". Este catálogo ofrece una relación de edificios artísticos de la ciudad de Madrid, con datos de dirección y de localización para georeferenciarlos. No se trata de una relación exhaustiva, sino de aquellos monumentos y edificios de los cuales ya figura información en la web municipal. El siguiente trozo de código permite recuperar el archivo xml en cuestion, y lo almacena en disco.

In [9]:
import urllib.request
x = urllib.request.urlopen('https://datos.madrid.es/egob/catalogo/208844-0-monumentos-edificios.xml')
a=x.read()
a=a.decode("utf-8")
prueba=open("catalogo.xml","w")
prueba.write(str(a))
prueba.close()

__[Ejercicio 1 [3 puntos]]__

Se pide mostrar al usuario un listado de los nombres de los monumentos que contiene el archivo con el objetivo de que elija uno de los monumentos. Para facilitar la selección del monumento se puede asignar a cada monumento un número. Fíjate que el nombre que se quiere recuperar es aquel que aparece dentro de las etiquetas <atributo nombre="NOMBRE">. Por ejemplo en el caso del "Banco de España" sería:
    
    <contenido>
        <tipo>EntidadesYOrganismos</tipo>
        <atributos idioma="es">
            <atributo nombre="ID-ENTIDAD">13947</atributo>
            <atributo nombre="NOMBRE">Banco de España</atributo>
            
A continuación se le mostrará por pantalla toda la información acerca del monumento de una manera amigable (es decir sin etiquetas).

Para la segund parte se va a utilizar la API de la __EMT__. La __EMT__ dispone de un portal de datos abiertos que se puede encontrar en la página http://opendata.emtmadrid.es/Home. El objetivo del mismo es ofrecer diferentes tipos de datos sobre la actividad de los autobuses de la EMT. El acceso a los datos se realiza a través de una API de servicios web. 

Para poder utilizar la API de servicios, en primer lugar hay que registrarse, lo cual puede hacerse en la página http://opendata.emtmadrid.es/Formulario. El registro devuelve como resultado en un mensaje electrónico, dos valores:

* Id de cliente: Identificador de cliente

* Pass Key: Password

A continuación, hay que elegir el servicio que se quiere utilizar. Hay 4 servicios definidos: BUS, GEO, MEDIA, INFOPARKING y BICIMAD. Cada servicio tiene asociado un conjunto de métodos que al invocarlos, devuelven un resultado. Por ejemplo, el servicio BUS dispone del servicio __"GetRouteLines"__ que obtiene el itinerario de una línea (o varias líneas separadas por el carácter pipe(|)), con los vértices para construir las rectas del recorrido y las coordenadas UTM de los ejes viales y los códigos de parada. Si se quiere invocar desde Python, se puede hacer usando el siguiente código:

En esta práctica se va a trabajar con varios métodos:

* Método __'GetStreet'__ del servicio GEO. Este método obtiene una lista de emplazamientos EMT coincidentes con una localización. Cada emplazamiento está compuesto por una lista de paradas EMT situadas dentro de un radio predefinido con todos sus atributos, así como las líneas EMT que pasan por cada parada de la lista. En el documento __Servicios_GEO.pdf__ adjunto a esta práctica, se explica con detalle los parámetros de entrada y el resultado que devuelve este método. El método se puede invocar indicando al menos el nombre de la calle y el número:

El método devuelve un documento XML. Entre los elementos del documento, se encuentran las coordenadas X e Y del lugar buscado. En la búsqueda del ejemplo, la calle búscada tiene como coordenadas UTM X e Y:

* Método __'GetStreetRoute'__ del servicio MEDIA. Este método obtiene hasta tres rutas óptimas para ir de un sitio a otro a pie y en autobús. En el documento __Servicios_MEDIA.pdf__ adjunto a esta práctica, se explica con detalle los parámetros de entrada y el resultado que devuelve este método. El método se puede invocar indicando los siguientes parámetros

  * idClient: Código de cliente autorizado para la operación y suministrado por EMT.

  * passKey: Código de clave asociado al cliente.

  * coordinateXFrom: Campo para identificar la posición "x" del origen de la ruta a consultar. 
  
  * coordinateYFrom: Campo para identificar la posición "y" del origen de la ruta a consultar.

  * coordinateXTo: Campo para identificar la posición "x" del destino de la ruta a consultar. 
  
  * coordinateYTo: Campo para identificar la posición "y" del destino de la ruta a consultar. 

  * criteriaSelection: Campo que indica el criterio de la búsqueda. Los valores son:
  
       * 11 - Mínimo tiempo de trayecto
    
       * 13 - Mínimos trasbordos
    
       * 14 - Mínimo recorrido a pie

El método devuelve un documento XML que contiene información sobre las rutas óptimas (se explica con detalle en el documento __Servicios_MEDIA.pdf__). En particular muestra la siguiente información:

* El elemento __DescriptionRouteData__ donde aparece la siguiente información:

  * DescriptionDate: Fecha de la ruta.
  * DescriptionInitTime: Descripción de hora de inicio de la ruta.
  * DescriptionEstimateTimeArrival: Descripción de la hora estimada de llegada.
  * Transfers: Trasbordos.
  * LongJourney: Duración del viaje.

* El elemento __ListSectionRoute__ aparece una lista de subelementos __Section__.Cada uno de ellos, describe parcialmente una parte de la ruta. En un caso ideal, aparecerá un subelemento de __Section__ de tipo __WalkingLeg__  que describe el inicio de la ruta andando, a continuación un conjunto de subelementos de __Section__ de tipo __BusLeg__ que describen cada uno de ellos la ruta en una línea de autobus que forma parte de la ruta buscada, y por último otro subelemento de __Section__ de tipo __WalkingLeg__  que describe el final de la ruta andando.

* El elemento __POI__ que muestra información sobre puntos de interés que se encuentran a lo largo de la ruta.

Se pide:

 __[Ejercicio 2 [2 puntos]]__

Crear una función que dada una calle de Madrid, devuelva las coordenadas X e Y de la calle.

 __[Ejercicio 3 [5 puntos]]__

Usando la función anterior completa el ejercicio 1. Para ello, se pide crear una función que solicite al usuario una calle de Madrid y obtenga la ruta más óptima para ir desde esa calle al monumento seleccionado por el usuario usando autobuses de la EMT. La función deberá imprimir por pantalla, la siguiente información:

 * Fecha de la ruta.
 
 * Hora de inicio de la ruta.
 
 * Hora estimada de llegada
 
 * Número de trasbordos.
 
 * Duración del viaje.
 
 * Descripción textual de la ruta.
 
 Usando el ejemplo de salida sería:
 
 * Fecha de la ruta: 26/10/2017
 
 * Hora de inicio de la ruta: 03:55
 
 * Hora estimada de llegada: 05:28
 
 * Número de trasbordos: 1
 
 * Duración del viaje: 93
 
 * Descripción textual de la ruta:
 
     * Caminar 42' hasta parada 3125 - EMILIO VARGAS-ARTURO SORIA, linea N4
     
     * 28' en autobús (línea N4) hasta parada 449 - SERRANO-ORTEGA Y GASSET
     
     * Desde parada 449 caminar 5' hasta parada 61 - Castellana-Ministerio Interior, linea N25
 
     * 15' en autobús (línea N25) hasta parada 3691 - SOL-SEVILLA

     * Desde parada 3691 caminar 3'  

#### Normas de entrega

* Fecha tope de entrega: 18/10/2018
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre FormatosII_GrupoX donde X será el número de grupo correspondiente.